![](https://images.pexels.com/photos/507410/pexels-photo-507410.jpeg?auto=compress&cs=tinysrgb&dpr=2&h=650&w=940)

# Apriltag VR

Kevin J. Walchko, Phd

18 July 2020

---

Apriltags provide the ability to produce virtual 

![](a.png)

![](b.jpg)

![](c.png)



## References

- Rosebrock: [Find distance from camera to object/marker using Python and OpenCV](https://www.pyimagesearch.com/2015/01/19/find-distance-camera-objectmarker-using-python-opencv/)
- [iPhone Calibration](https://medium.com/@tomas789/iphone-calibration-camera-imu-and-kalibr-33b8645fb0aa)

In [1]:
# reload library
%load_ext autoreload
%autoreload 2

# plotting
%matplotlib inline 

In [12]:
%matplotlib inline
import numpy as np
np.set_printoptions(precision=1)
np.set_printoptions(suppress=True)

from matplotlib import pyplot as plt
import pylab                                 
pylab.rcParams['figure.figsize'] = (10.0, 18.0)

import time

from pupil_apriltags import Detector
import apriltag_gen as apt

import imageio
import cv2

## Helper Functions

In [13]:
def draw_tag(color_img, corners, tag_id=None):
    """
    color_img: image to draw on, must be color
    corners: corner points from apriltag detector, v[0] is the 
             lower left of the tag and the point move CCW.
    """ 
    pts = corners.reshape((-1,1,2)).astype('int32')
    cv2.polylines(color_img,[pts],True,(0,255,0),thickness=4)

    y = color_img.shape[0]
    r = max(int(y/200),1)
    c = (255,0,0)
    oc = (0,0,255)
    v = corners.astype('int32')
    cv2.circle(color_img, tuple(v[0]),r,oc,thickness=-1)
    cv2.circle(color_img, tuple(v[1]),r,c,thickness=-1)
    cv2.circle(color_img, tuple(v[2]),r,c,thickness=-1)
    cv2.circle(color_img, tuple(v[3]),r,c,thickness=-1)
    
    sz = np.abs(v[0] - v[2])
    sz = sz[0]*sz[1]
    
    if tag_id is not None:
        offset = int((v[1][0]-v[0][0])/4)
        fontScale = sz / (100 * 100 * 3)
        cv2.putText(color_img, str(tag_id),
                    org=(v[0][0]+offset,v[0][1]-offset,),
                    fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                    fontScale=fontScale,
                    thickness=4,
                    color=(255, 0, 255))
    
    return color_img

def draw_id(img, tags):
    """
    Draws the tag IDs on the found tags
    
    img: single color image
    tags: array of tags found in image
    """
    if len(img.shape) == 2:
        color_img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    else:
        color_img = img.copy()

    for tag in tags:
        color_img = draw_tag(color_img, tag.corners, tag.tag_id)
    return color_img

def draw_imgpts(img, tags):
    if len(img.shape) == 2:
        color_img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    else:
        color_img = img.copy()

    for t in tags:
            color_img = draw_tag(color_img, t.corners)
        
    return color_img